In [ ]:
from datetime import datetime, date
import pandas as pd
from gs_quant.instrument import IRSwap
from gs_quant.backtests.triggers import PeriodicTrigger, PeriodicTriggerRequirements
from gs_quant.backtests.actions import HedgeAction
from gs_quant.backtests.generic_engine import GenericEngine
from gs_quant.backtests.strategy import Strategy
from gs_quant.risk import Price, IRDelta

In [ ]:
# Initialize session
from gs_quant.session import GsSession

GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

## Inflation strategy hedging a 50y inflation swap with a 30y inflation swap.

In [ ]:
# Define backtest dates
start_date = date(2024, 1, 3)
end_date = datetime.today().date()

In [ ]:
# Define instruments for strategy

#  initial holding of 50y swap hedge with daily 30y swaps where the CMS of the main swap and the hedge are different
swap_hedge = IRSwap(
    pay_or_receive='Receive',
    termination_date='30y',
    notional_currency='EUR',
    notional_amount=100e6,
    fixed_rate='ATM',
    name='30yhedge',
)
swap = IRSwap(
    pay_or_receive='Pay',
    termination_date='50y',
    notional_currency='EUR',
    notional_amount=100e6,
    fixed_rate='ATM',
    name='50y',
)

In [ ]:
# Periodic trigger: based on frequency
freq = "1b"

trig_req = PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency=freq)
actions_hedge = HedgeAction(IRDelta(aggregation_level='Type'), swap_hedge, csa_term='EUR-OIS')
triggers = PeriodicTrigger(trig_req, actions_hedge)
strategy = Strategy([swap], triggers)

# run backtest daily
GE = GenericEngine()
backtest = GE.run_backtest(strategy, start=start_date, end=end_date, frequency='1b', show_progress=True)

In [ ]:
# View backtest trade ledger
backtest.trade_ledger()

In [ ]:
# View results summary
backtest.result_summary

In [ ]:
# View Mark to Market
pd.DataFrame({'Generic backtester': backtest.result_summary[Price]}).plot(figsize=(10, 6), title='Mark to market')

In [ ]:
# View Performance
pd.DataFrame({'Generic backtester': backtest.result_summary['Cumulative Cash'] + backtest.result_summary[Price]}).plot(
    figsize=(10, 6), title='Performance'
)